In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk

import math
import json
import re
import string

In [2]:
# load in the textual data from the excel file
df_data = pd.read_excel("../data/dvlog_transcripts.xlsx")
df_data.head()

,video_id,label,key,gender,duration,channelId,transcript,transcript annotated
0,0,depression,2s3EFyjUmfs,f,823.31,UCWx_Fqt2AziUyAHVjgUH1PQ,so I wanted to come on here and sit down with ...,[{'text': 'so I wanted to come on here and sit...
1,1,depression,MCaKQvEofrE,f,436.65,UCxxhKhiRPfPe_U78ao3FKLw,hello guys it's me again and i'm going to talk...,"[{'text': 'hello guys', 'start': 9.599, 'durat..."
2,2,depression,6o-DlNR_cIs,m,835.83,UCCXKRG3s-auYMFIAKjeey_Q,welcome back to another video today I'm gonna ...,"[{'text': ""welcome back to another video today..."
3,3,depression,gxZjgt3Gnug,m,420.61,UCuhfaNXfRLFfVVc6dSNoyhA,hi everybody and welcome to and in the clouds ...,[{'text': 'hi everybody and welcome to and in ...
4,4,depression,oc72xdTxJ50,f,444.55,UCbiKXPwk590XYYqqCEbtgoA,"Hey, Hey Me and my crown Me and my hairy hair ...","[{'text': 'Hey, Hey', 'start': 1.04, 'duration..."


In [3]:
def format_transcript(transcript) -> list[dict]:

    # add the top property name
    json_text = '{"result": ' + transcript + '}'

    # replace the escape characters for the keys
    json_text = json_text.replace("\'text\'", '"text"').replace("\'start\'", '"start"').replace("\'duration\'", '"duration"')
    
    # replace the escape characters at the beginning and end
    json_text = json_text.replace(" \'", '"').replace("\',", '",')

    # parse the text using json
    parsed_data = json.loads(json_text)

    # return the formatted transcripts
    return parsed_data["result"]
    

In [10]:
def clean_text(text: str) -> str:
    ...
    # Remove all numbers with lookup table
    text = text.translate(str.maketrans("", "", string.digits))

    # remove special symbols
    text = re.sub("[^a-zA-Z0-9 \n\.]", "", text)

    # remove punctuations with Lookup table
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Remove excess whitespace in between words
    text = " ".join(text.split())
    return text



def text_to_seconds(transcripts: list[dict], video_time: float) -> dict:
    """"""
    # 
    seconds_dict = {i: [] for i in range(0, math.floor(video_time + 1))}

    # setup the variables
    start_time = None
    end_time = None

    # handle the transcripts in between
    for i, entry in enumerate(transcripts[0:-1]):
        # get the start and end time
        start_time = entry["start"]
        end_time = transcripts[i+1]["start"]

        # 
        cleaned_text = clean_text(entry["text"])
        tokens = cleaned_text.split()
        cues = np.floor(np.linspace(start_time, end_time, num=len(tokens)))

        # put each token in the correct bucket
        for token, cue in zip(tokens, cues):
            seconds_dict[cue].append(token)

    # handle the last transcript
    tokens = clean_text(transcripts[-1]["text"]).split()
    cues = np.floor(np.linspace(end_time, video_time, num=len(tokens)))

    # put the last tokens in the buckets
    for token, cue in zip(tokens, cues):
        seconds_dict[cue].append(token)

    # return the buckets
    return seconds_dict

test = [{'text': 'so I wanted to come on here and sit down',
  'start': 0.03,
  'duration': 4.289},
 {'text': 'with you guys and just kind of talk and',
  'start': 2.46,
  'duration': 4.919},
 {'text': 'vent because I have been really', 'start': 4.319, 'duration': 6.36},
 {'text': 'struggling lately with this whole TTC',
  'start': 7.379,
  'duration': 7.081},
 {'text': 'journey and I have gotten to a point in',
  'start': 10.679,
  'duration': 7.43},
 {'text': "this journey where I'm obsessed and",
  'start': 14.46,
  'duration': 5.94}]

# text = df_data["transcript annotated"].iloc[0]
# new_text = format_transcript(text)
# text_to_seconds(test, 16)
text = df_data["transcript annotated"].iloc[4]
text

'[{\'text\': \'Hey, Hey\', \'start\': 1.04, \'duration\': 0.56}, {\'text\': \'Me and my crown\', \'start\': 5.38, \'duration\': 0.96}, {\'text\': \'Me and my hairy hair (arm*) pits\', \'start\': 6.34, \'duration\': 1.58}, {\'text\': "My flat\'s a mess", \'start\': 11.42, \'duration\': 0.96}, {\'text\': "I\'m a mess. My head\'s a mess.", \'start\': 13.34, \'duration\': 1.84}, {\'text\': \'So, step one to feel better Take care of your mess.\', \'start\': 21.78, \'duration\': 3.28}, {\'text\': \'Step 1.fun\', \'start\': 30.02, \'duration\': 0.72}, {\'text\': \'Pick out good music\', \'start\': 32.72, \'duration\': 1.28}, {\'text\': \'Step 1. ick\', \'start\': 43.32, \'duration\': 0.72}, {\'text\': \'Little tasks\', \'start\': 46.6, \'duration\': 0.88}, {\'text\': "I\'m gonna clean up and tidy up for ten minutes straight. That ain\'t much", \'start\': 50.1, \'duration\': 4.72}, {\'text\': "That\'s small. I can do that.", \'start\': 54.82, \'duration\': 1.38}, {\'text\': \'I can do ten.\', 

In [9]:
# 
formatted_transcripts = {}

# 
for index, row in df_data.iterrows():
    # 
    print(index)
    text, duration = row["transcript annotated"], row["duration"]
    new_text = format_transcript(text)
    new_format = text_to_seconds(new_text, duration)

    # save it all
    formatted_transcripts[index] = new_format

0
1
2
3
4


JSONDecodeError: Expecting ',' delimiter: line 1 column 3150 (char 3149)